Problem:  
You and some friends are deciding teams for a game. Some people don't like each other, and don't want to be on the same team. There are also people who need to be on the same team because they are such good friends. There are also people who don't mind if someone is on their team or not.  
Edge types:  
* Friend: Must be on the same team
* Enemy: Must be on different teams
* Neutral: Doesn't matter if on same or different team

In [1]:
from my_libraries.graph_theory.MyGraph import *
import random,math

In [2]:
def randind(arr):
    return math.floor(random.random()*len(arr))
def getRelNeighbors(G,node,rel):
    ans = set([])
    for n in G.get_neighbors(node):
        if G[node]['to'][n]['rel'] is rel:
            ans.add(n)
    return ans

## prepare graph

In [3]:
players = [
    'mike',
    'bobby',
    'eubihn',
    'feit',
    'austin',
    'kibby',
    'jonah',
    'sean'
]
numTeams = 2
G = Graph()
G.add_node(*players)
G

{'austin': {'from': {}, 'to': {}},
 'bobby': {'from': {}, 'to': {}},
 'eubihn': {'from': {}, 'to': {}},
 'feit': {'from': {}, 'to': {}},
 'jonah': {'from': {}, 'to': {}},
 'kibby': {'from': {}, 'to': {}},
 'mike': {'from': {}, 'to': {}},
 'sean': {'from': {}, 'to': {}}}

In [4]:
G.set_edge('mike','feit',{'rel':'enemy'})
G.set_edge('mike','bobby',{'rel':'friend'})
G.set_edge('austin','jonah',{'rel':'friend'})
G.set_edge('eubihn','jonah',{'rel':'enemy'})
G.set_edge('feit','kibby',{'rel':'enemy'})
G

{'austin': {'from': {'jonah': {'rel': 'friend'}},
  'to': {'jonah': {'rel': 'friend'}}},
 'bobby': {'from': {'mike': {'rel': 'friend'}},
  'to': {'mike': {'rel': 'friend'}}},
 'eubihn': {'from': {'jonah': {'rel': 'enemy'}},
  'to': {'jonah': {'rel': 'enemy'}}},
 'feit': {'from': {'kibby': {'rel': 'enemy'}, 'mike': {'rel': 'enemy'}},
  'to': {'kibby': {'rel': 'enemy'}, 'mike': {'rel': 'enemy'}}},
 'jonah': {'from': {'austin': {'rel': 'friend'}, 'eubihn': {'rel': 'enemy'}},
  'to': {'austin': {'rel': 'friend'}, 'eubihn': {'rel': 'enemy'}}},
 'kibby': {'from': {'feit': {'rel': 'enemy'}},
  'to': {'feit': {'rel': 'enemy'}}},
 'mike': {'from': {'bobby': {'rel': 'friend'}, 'feit': {'rel': 'enemy'}},
  'to': {'bobby': {'rel': 'friend'}, 'feit': {'rel': 'enemy'}}},
 'sean': {'from': {}, 'to': {}}}

In [5]:
for a in G.get_nodes():
    for b in G.get_nodes():
        if b not in G.get_neighbors(a) and a is not b:
            G.set_edge(a,b,{'rel':'neutral'})
G['mike']

{'from': {'austin': {'rel': 'neutral'},
  'bobby': {'rel': 'friend'},
  'eubihn': {'rel': 'neutral'},
  'feit': {'rel': 'enemy'},
  'jonah': {'rel': 'neutral'},
  'kibby': {'rel': 'neutral'},
  'sean': {'rel': 'neutral'}},
 'to': {'austin': {'rel': 'neutral'},
  'bobby': {'rel': 'friend'},
  'eubihn': {'rel': 'neutral'},
  'feit': {'rel': 'enemy'},
  'jonah': {'rel': 'neutral'},
  'kibby': {'rel': 'neutral'},
  'sean': {'rel': 'neutral'}}}

## random guess and check method
problems:
* indefinite performance time
* if a correct arrangement does not exist, the program will infinitely loop

In [6]:
def guess():
    teams = []
    players = list(G.get_nodes())
    while len(players) > 0:
        team = []
        while len(team)< math.floor(len(G.get_nodes())/numTeams) and len(players)>0:
            team.append(players.pop(randind(players)))
        teams.append(team)
    return teams
guess()

[['mike', 'austin', 'jonah', 'kibby'], ['sean', 'eubihn', 'bobby', 'feit']]

In [7]:
def check(guess):
    #check enemies
    for team in guess:
        for a in team:
            for b in team:
                if a is b:
                    continue
                else:
                    if G[a]['to'][b]['rel'] is 'enemy':
                        return False
    #check friends
    for team in guess:
        for a in team:
            friends = getRelNeighbors(G,a,'friend')
            if not friends <= set(team):
                return False
    return True
g = guess()
while not check(g):
    g = guess()
g

[['eubihn', 'kibby', 'mike', 'bobby'], ['feit', 'jonah', 'sean', 'austin']]

## lexicographic guess and check method

In [8]:
from my_libraries.lexicographicOrder import *

In [18]:
def guess(perm):
    teams = []
    players = perm[:]
    while len(players) > 0:
        team = []
        while len(team)< math.floor(len(G.get_nodes())/numTeams) and len(players)>0:
            team.append(players.pop(0))
        teams.append(team)
    return teams
guess(list(G.get_nodes())),list(G.get_nodes())

([['kibby', 'jonah', 'bobby', 'feit'], ['mike', 'eubihn', 'austin', 'sean']],
 ['kibby', 'jonah', 'bobby', 'feit', 'mike', 'eubihn', 'austin', 'sean'])

In [21]:
# uses the same check function as random guess and check method
players = list(G.get_nodes())
for perm in allPerms(players):
    perm = list(perm)
#     print(guess(perm))
    if check(guess(perm)):
        print(perm,guess(perm))
        break

['kibby', 'bobby', 'mike', 'eubihn', 'jonah', 'feit', 'austin', 'sean'] [['kibby', 'bobby', 'mike', 'eubihn'], ['jonah', 'feit', 'austin', 'sean']]


## direct method

In [11]:
def dft(G,a,rel):
    visited = set([a])
    def f(a):
        visited.add(a)
        for b in getRelNeighbors(G,a,rel):
            if b not in visited:
                f(b)
    f(a)
    return visited
dft(G,'eubihn','enemy')

{'eubihn', 'jonah'}